# Check Files


In [ ]:
import os
import pandas as pd
import shutil
import numpy as np
from unicodedata import normalize, category
import re

In [2]:
# Data source where I get the raw data
source_path = 'C:\\zomato\\source'
# Folder where I move the files after processed 
processed_path='C:\\zomato\\processed\\'
# Result folder
result_path='C:\\zomato\\result\\'
# Areas dataset
areas_path='C:\\zomato\\source\Areas_in_blore.xlsx'

In [3]:
# Lists to receive informations
processed_files = []
source_files = []
validated_file = []

In [4]:
# Check if is csv and loading in the list source_csv_files
def check_csv():
    for csv_file in os.listdir(source_path):
        if csv_file.endswith(".csv"):
            source_files.append(csv_file)
check_csv()

In [5]:
# Get in processed folder the files name
def check_processed():
    for processed_file in os.listdir(processed_path):
        processed_files.append(processed_file)
check_processed()

In [6]:
# Check in the processed folder if the day's files have already been processed
difference = set(processed_files).difference(set(source_files))
difference_2 = set(source_files).difference(set(processed_files))
list_difference = list(difference.union(difference_2))
print(list_difference)

['data_file_20210528182844.csv', 'data_file_20210527182730.csv', 'data_file_20210527182731.csv', 'data_file_20210528182554.csv']


In [7]:
# Check if files are empty
def check_empty():
    for nempty_file in list_difference:
        mount_file = source_path + "\\" + nempty_file
        
        if (os.stat(mount_file).st_size > 2):
            validated_file.append(mount_file)
            print(mount_file)
check_empty()

C:\zomato\source\data_file_20210528182844.csv
C:\zomato\source\data_file_20210527182730.csv
C:\zomato\source\data_file_20210528182554.csv


# Data Trans


In [8]:
# Remove especial/junk characters from the text. 
def Remove_Sc(get):
    run = get.replace("ñ", "n").replace("Ñ", "N")
    run = re.sub("\\\\n", " ",run)
    #run = re.sub('[^A-Za-z0-9]+', ' ', get)
    run_final = normalize("NFKD", run).encode("ascii","ignore").decode("ascii")
    
    # Replace special chars including ( )
    run_final=re.sub("[^(\w|\d|\s)]|\?|x(\d){2}(A|E|I|O|U)","", run_final)  
    return re.sub("(A|E|I|O|U){2,20}|(A|E|I|O|U)x(\d){2}(A|E|I|O|U)","", run_final)


In [9]:
for execution in validated_file:
    # Dataset with specified columns
    df = pd.read_csv(execution, usecols = ['url','address','name','rate','votes','phone','location',
                                            'rest_type','dish_liked','cuisines','reviews_list'])
    # Dataset to be used for check null values
    df_check_error = df
    
    # mounting the target path
    df_out = re.sub('.csv','.out',result_path+os.path.basename(execution))
    df_bad = re.sub('.csv','.bad',result_path+os.path.basename(execution))
    df_meta = re.sub('.csv','.metadata',result_path+os.path.basename(execution))
    
    # creating flag for null values in the df_check_error to be use as a filter
    df_check_error.loc[df_check_error['phone'].isnull(),'value_is_NaN_phone'] = 'Yes'
    df_check_error.loc[df_check_error['phone'].notnull(), 'value_is_NaN_phone'] = 'No'
    df_check_error.loc[df_check_error['name'].isnull(),'value_is_NaN_name'] = 'Yes'
    df_check_error.loc[df_check_error['name'].notnull(), 'value_is_NaN_name'] = 'No'
    df_check_error.loc[df_check_error['location'].isnull(),'value_is_NaN_locat'] = 'Yes'
    df_check_error.loc[df_check_error['location'].notnull(), 'value_is_NaN_locat'] = 'No'
    
    #validations dataset filtering null values to load in the file .bad and metadata
    df_phone_null = df_check_error[df_check_error["value_is_NaN_phone"] == 'Yes']
    df_name_null = df_check_error[df_check_error["value_is_NaN_name"] == 'Yes']
    df_location_null = df_check_error[df_check_error["value_is_NaN_locat"] == 'Yes']

# address,reviews_list, name     
    # Remove especial/junk characters from the text
    # apply the function in each element in the list (comlumns)
    df['name']= df['name'].map(Remove_Sc)
    df['reviews_list']= df['reviews_list'].map(Remove_Sc)
    df['address']= df['address'].map(Remove_Sc)

# Phone 
    # 'n' and 'na' to -1 in order to change the field type to int. considering 'n' and 'na' as not applicable
    df['phone'] = df['phone'].replace('n', "-1").replace('na', "-1").replace('', '-1')
    
    # Split the phone into two columns when necessary    
    df[['contact_number_1', 'contact_number_2']] = df['phone'].str.split('\r\n |\n', n=1, expand=True)
    
    # Replace + and spaces
    df['contact_number_1'] = df['contact_number_1'].apply(lambda a: str(a).replace('+','')).apply(lambda a: str(a).replace(' ','')).apply(lambda a: str(a).replace('\r',''))
    df['contact_number_2'] = df['contact_number_2'].apply(lambda a: str(a).replace('+','')).apply(lambda a: str(a).replace(' ',''))
    
# Validation Areas
    #Validation of location field for correctness of data by looking up to the Areas_in_blore.csv file.
    df_area = pd.read_excel(areas_path,  engine='openpyxl')
    df['check_area'] = df.location.isin(df_area.Area).astype(int)  
    df_out_area = df.loc[df['check_area'] == 1]
    df_bad_area = df.loc[df['check_area'] == 0]
    
    df_bad_address = df_bad_area[df_bad_area["check_area"] == 0]
    
    #df_bad_address = df_bad_area
    df_bad_area.index.names = ['Row_num_list']
    
# Write .bad and metadata whit trash datas
    # check If the dataframe has bad_area
    if(df_bad_area['name'].count() >= 1):
            #load bad data to .bad and problem types in metadata file
            df_bad_address.to_csv(df_bad,index=0)
            df_bad_area['Type_of_issue'] = 'Invalid Adress'
            df_bad_area['Type_of_issue'].to_csv(df_meta)
            
    if (df_phone_null['value_is_NaN_phone'].count() >= 1):
            ##Capturing all the bad records to a .bad & metadata file.
            df_phone_null.to_csv(df_bad, mode='a', header=False, index=1)
            df_phone_null['Type_of_issue']="phone is null"
            df_phone_null['Type_of_issue'].to_csv(df_meta, mode='a', header=False,index=1)
            
    if (df_name_null['value_is_NaN_name'].count() >= 1):
            ##Capturing all the bad records to a .bad & metadata file.
            df_name_null.to_csv(df_bad, mode='a', header=False, index=1)
            df_name_null['Type_of_issue']="Name is null"
            df_name_null['Type_of_issue'].to_csv(df_meta, mode='a', header=False,index=1)
            
    if (df_location_null['value_is_NaN_locat'].count() >= 1):
            ##Capturing all the bad records to a .bad & metadata file.
            df_location_null.to_csv(df_bad, mode='a', header=False, index=1)
            df_location_null['Type_of_issue']="location is null"
            df_location_null['Type_of_issue'].to_csv(df_meta, mode='a', header=False,index=1)   
            

    # set columns that cannot have null values
    df_out_area = df_out_area.loc[df['name'].notnull()]
    df_out_area = df_out_area.loc[df['phone'].notnull()]
    df_out_area = df_out_area.loc[df['location'].notnull()]    
    
    #change type
    df_out_area['contact_number_1'] = df_out_area['contact_number_1'].astype('int64')
    
    df_final_out = df_out_area[['url','address','name','rate','votes','contact_number_1','contact_number_2','location',
                                            'rest_type','dish_liked','cuisines','reviews_list']]
    
    # Load the out file
    df_final_out.to_csv(df_out,index=0)
    
    #Copy the processed files in the processor folder to be used in the verification of new files
    shutil.copyfile(execution, processed_path+os.path.basename(execution))
    


<ipython-input-9-821a707b826b>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_area['Type_of_issue'] = 'Invalid Adress'
<ipython-input-9-821a707b826b>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phone_null['Type_of_issue']="phone is null"
<ipython-input-9-821a707b826b>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-